The examples are brought from the paper of the Japanese.

The other one is of my doing.

In [9]:

from openai import OpenAI
import pandas as pd
client = OpenAI()

df = pd.read_csv('id2.csv')

def meaning(id):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You will be provided with an idiom in Moroccan Darija dialect, "+
            "and your task is to determine its possible meaning."},
           {"role": "user", "content": "طلع تاكول الكرموس, نزل شكون قالها ليك."},
            {"role": "assistant", "content": "The idiom is about someone who's been told to do something but then he gets blamed after he does it."},
            {"role": "user", "content":  f"{id}" }
            ], 
        temperature=0
        )
    return completion.choices[0].message.content


def ohne(id):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You will be provided with an idiom in Moroccan Darija dialect, "+
            "and your task is to translate it into an equivalent idiom or construction in English with no explanations."},
            {"role": "user", "content":  f"{id}" }
            ], 
        temperature=0
        )
    return completion.choices[0].message.content

def culture(id):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Translate the provided Moroccan Darija dialect idiom into English. "+
            "Please fulfill the following conditions when translating.\n"+
            "Purpose of the translation: Use natural expressions that can be "+
            "understood by English speakers who are not very familiar with Moroccan culture.\n"+
            "Target audience: General English-speaking audience."},
            {"role": "user", "content":  f"{id}" }
            ], 
        temperature=0
        )
    return completion.choices[0].message.content


def dynamic(id):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": 
             "Dynamic equivalence is a strategy for translating from the perspective of equalizing the "+
            "reader’s response to the [source text] and the [target text]. In the example below, the word "+
            "“Lamb” in the original text would be translated as “lamb” in the literal translation. However, "+
            "when translating for Iceland, which has no sheep, it is difficult to convey the nuance of the "+
            "word “lamb”. From the standpoint of equalizing the reader’s reaction, this is a ruse to translate "+
            "it as “seal”. It is believed that “lamb” in the [source text] and “seal” in the [target text]"+
            "will evoke the same reaction in the reader.\n"+
            "[source text] Lamb of God\n"+
            "[target text] Seal of God\n"+
            "Following this concept and example, please translate the following [source text] into English "+
            "using the dynamic equivalent. Please replace the translation with something that would be "+
            "understood in an English-speaking culture."
             },
            {"role": "user", "content":  f"{id}" }
            ], 
        temperature=0
        )
    return completion.choices[0].message.content

def multiple_turns(id):
    literal_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in Language translation from Moroccan Darija dialect to English. " +
            "You will be provided with a sentence in Moroccan Darija dialect, "+
            "and your task is to translate it into an English sentence. "+
            "The translation must be literal with no explanations."},
            #{"role": "assistant", "content": "OK, I understand. I am ready for the translation."},
            {"role": "user", "content": "طلع تاكول الكرموس, نزل شكون قالها ليك."},
            {"role": "assistant", "content": "Come upstairs and have some figs, get downstairs who told you so!"},
            {"role": "user", "content":  f"{id}" }
        ], 
        temperature=0  # Deterministic responses
    )
    literal = literal_completion.choices[0].message.content

    # Step 1: Conclude a possible explanation for the given Darija idiom
    explanation_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
            "You will receive a literal English translation of a Moroccan Darija idiom, "+
            "and your task is to determine its possible meaning."},
            {"role": "user", "content": "Come upstairs and have some figs, get downstairs who told you so!"},
            {"role": "assistant", "content": "The idiom is about someone who's been told to do something but then he gets blamed after he does it."},

            {"role": "user", "content": f"{literal}" }
        ], 
        temperature=0  # Deterministic responses
    )
    explanation = explanation_completion.choices[0].message.content
    
    # Step 2: Give an equivalent idiom from the explanation
    equivalent_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
            "You will be provided with an explanation of an idiom, "+
            "and your task is to provide the equivalent English idiom or construction for it."},
            {"role": "user", "content": "The idiom is about someone who's been told to do something but then he gets blamed after he does it."},
            {"role": "assistant", "content": "Caught between a rock and a hard place"},
            {"role": "user", "content": f"{explanation}" }
        ], 
        temperature=0  # Deterministic responses
    )
    equivalent_idiom = equivalent_completion.choices[0].message.content
    
    # Step 3: Revise output of the translation before outputting it
    revision_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
            "You will receive a possible English idiom, " +
            "and your task is to revise it for accuracy and appropriateness. " +
            "If it fits as an idiom, keep it; if not, provide a suitable alternative."},
            {"role": "user", "content": "Caught between a rock and a hard place."},
            {"role": "assistant", "content": "Caught between a rock and a hard place."},
            {"role": "user", "content": f"{equivalent_idiom}" }
        ], 
        temperature=0  # Deterministic responses
    )
    revised_idiom = revision_completion.choices[0].message.content
    
    return literal, explanation,equivalent_idiom,revised_idiom


In [11]:
darija_idiom = "أش خاصك يا العريان, خاصني خاتم الذهب يا مولاي."
print("Literal translation: You! The one naked, what do you need? I need a ring made of gold my lord.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")

Literal translation: You! The one naked, what do you need? I need a ring made of gold my lord.

prompt's literal translation: What do you want, you naked one, give me the gold ring, sir.
prompt's explanation of original idiom in English: This idiom means "Mind your own business" in English.

Simple equivalent translation: What do you want, O naked one, give me the golden ring, my lord.
Purpose and target translation: What do you want, you naked one? You want me to give you a gold ring, my lord.
Culture specefic translation: What's up, naked one? I want the gold ring, my lord.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is being nosy or interfering in someone else's business.
Second turn: equivalent of the meaning in English: To ask for the moon
Third turn: Recheck of the translation: To ask for the moon


In [10]:
darija_idiom = "جا يكحلها عماها."
print("Literal translation: Came to do her eyeliner, blinded her instead.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: Came to do her eyeliner, blinded her instead.

prompt's literal translation: He came to fix it for her.
prompt's explanation of original idiom in English: This idiom means that someone is trying to make a situation worse instead of improving it.

Simple equivalent translation: He came to put kohl on her eyes.
Purpose and target translation: He came to put kohl on her eyes.
Culture specefic translation: He hit the nail on the head.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is trying to help or solve a problem for someone else.
Second turn: equivalent of the meaning in English: To lend a helping hand
Third turn: Recheck of the translation: To lend a helping hand


In [17]:
darija_idiom = "اللي ما عندو فلوس كلامو مسوس."
print("Literal translation: If one doesn't have money, their speech is tasteless.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")

Literal translation: If one doesn't have money, their speech is tasteless.

prompt's literal translation: Whoever doesn't have money, their words are considered nonsense.
prompt's explanation of original idiom in English: This idiom means that someone who doesn't have money is not taken seriously or their words are not valued.

Simple equivalent translation: Empty vessels make the most noise.
Purpose and target translation: Someone who doesn't have money, their words are worthless.
Culture specefic translation: Broke as a joke.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that people who are not financially stable or successful are often not taken seriously or their opinions are not valued.
Second turn: equivalent of the meaning in English: "Money talks"
Third turn: Recheck of the translation: "Money talks"


In [19]:
darija_idiom = "امدح صاحبك مع الناس ولومو الراس فالراس"
print("Literal translation: Praise in public, criticize in private.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")

Literal translation: Praise in public, criticize in private.

prompt's literal translation: Praise your friend in front of people and scold him in private.
prompt's explanation of original idiom in English: This idiom means to praise your friend in front of others, even if you have to criticize him in private.

Simple equivalent translation: Praise your friend in front of people, but scold him in private.
Purpose and target translation: Praise your friend in front of others, even if you have to blame him in private.
Culture specefic translation: Praise your friend in public and blame the head, not the tail.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means to speak well of your friend in public and criticize them in private.
Second turn: equivalent of the meaning in English: "Speak of the devil and he shall appear."
Third turn: Recheck of the translation: To speak with a forked tongue


In [20]:
darija_idiom = "دير عين شافت وعين ماشافت"
print("Literal translation: One eye saw and the other eye did not see.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")

Literal translation: One eye saw and the other eye did not see.

prompt's literal translation: Put an eye that sees and an eye that doesn't see.
prompt's explanation of original idiom in English: This idiom means "to pretend not to see or know something."

Simple equivalent translation: Turn a blind eye.
Purpose and target translation: One eye saw, the other eye didn't.
Culture specefic translation: Seeing is believing

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means to be cautious and observant, to pay attention to details and be aware of your surroundings.
Second turn: equivalent of the meaning in English: Keep your eyes peeled
Third turn: Recheck of the translation: Keep your eyes peeled


In [21]:
darija_idiom = "اللي بغي لعسل يصبر لقريص النحل"
print("Literal translation: If one wants honey, they must endure the sting of bees.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: If one wants honey, they must endure the sting of bees.

prompt's literal translation: Whoever wants honey should be patient for the beeswax.
prompt's explanation of original idiom in English: This idiom means that if someone wants to achieve something good or valuable, they need to be patient and endure some difficulties or challenges along the way.

Simple equivalent translation: If you want honey, be prepared to deal with bees.
Purpose and target translation: Whoever wants honey must endure the sting of bees.
Culture specefic translation: He who wants honey must endure the sting of the bee.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that if you want to achieve something good or valuable, you need to be patient and willing to put in the necessary effort or work.
Second turn: equivalent of the meaning in English: Good things come to those who wait
Third turn: Recheck of the translation: Good things come to tho

In [15]:
darija_idiom = "اربط حمارك مع الحمير، يتعلم الشيق والنهيق وخرجان الطريق"
print("Literal translation: Tie your donkey with other donkeys, it will only learn to bray, neigh and from the path he'll stray.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: Tie your donkey with other donkeys, it will only learn to bray, neigh and from the path he'll stray.

prompt's literal translation: Tie your donkey with the other donkeys, it will learn the braying and neighing and get out of the way.
prompt's explanation of original idiom in English: This idiom means "Associate with those who are knowledgeable and experienced to learn and improve yourself."

Simple equivalent translation: Tie your donkey with the donkeys, it will learn how to bray and trot and stay on the road.
Purpose and target translation: Tie your donkey with the donkeys, it will learn the braying and the neighing and get out of the way.
Culture specefic translation: Tie your donkey with the donkeys, it learns the braying and the neighing and goes out of the way.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that when you spend time with a certain group of people, you start to adopt their habits and behaviors

In [22]:
darija_idiom = "الگنازة كبيرة والميت فار"
print("Literal translation: A grand funeral for a dead mouse.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: A grand funeral for a dead mouse.

prompt's literal translation: The funeral is big and the deceased is a mouse.
prompt's explanation of original idiom in English: This idiom means that the coffin is big but the deceased is small, implying that someone is making a big deal out of a small issue.

Simple equivalent translation: The coffin is big, but the deceased is a mouse.
Purpose and target translation: The funeral is grand, but the deceased is gone.
Culture specefic translation: The funeral is grand and the deceased is a mouse.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that a big fuss is being made over something insignificant or unimportant.
Second turn: equivalent of the meaning in English: Making a mountain out of a molehill
Third turn: Recheck of the translation: Making a mountain out of a molehill.


In [24]:

darija_idiom = "اللي تربي على شي حاجة كبر عليها"
print("Literal translation: If you are raised with something you grow old with it.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: If you are raised with something you grow old with it.

prompt's literal translation: Whoever is raised on something will grow accustomed to it.
prompt's explanation of original idiom in English: This idiom means that if someone is raised or brought up with a certain thing, they will eventually get used to it or become accustomed to it.

Simple equivalent translation: You can't teach an old dog new tricks.
Purpose and target translation: If you raise something, it will grow on you.
Culture specefic translation: Someone who is raised on something will eventually outgrow it.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that people tend to get used to whatever they are exposed to from a young age.
Second turn: equivalent of the meaning in English: You can't teach an old dog new tricks.
Third turn: Recheck of the translation: You can't teach an old dog new tricks.


In [25]:

darija_idiom = "لي حفر شي حفرة كيطيح فيها"
print("Literal translation: One who dugs a hole fells into it.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")



Literal translation: One who dugs a hole fells into it.

prompt's literal translation: Who digs a hole will fall into it.
prompt's explanation of original idiom in English: This idiom means "he who digs a hole will fall into it", which is similar to the English saying "you reap what you sow". It implies that someone who causes trouble or creates a problem for others will eventually face the consequences themselves.

Simple equivalent translation: He who digs a pit will fall into it.
Purpose and target translation: "He who digs a hole falls into it."
Culture specefic translation: He dug a hole and fell into it.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that if you do something wrong or harmful, you will likely face the consequences or negative outcomes of your actions.
Second turn: equivalent of the meaning in English: You reap what you sow
Third turn: Recheck of the translation: You reap what you sow.


In [27]:

darija_idiom = "الفم المسدود ما يدخلو دبان"
print("Literal translation: Flies do not enter a closed mouth.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")

Literal translation: Flies do not enter a closed mouth.

prompt's literal translation: A closed mouth doesn't let flies in.
prompt's explanation of original idiom in English: This idiom means that a closed mouth doesn't let flies in, implying that it's better to keep quiet and avoid trouble.

Simple equivalent translation: A closed mouth catches no flies.
Purpose and target translation: A blocked mouth doesn't let flies in.
Culture specefic translation: A closed mouth catches no flies.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that it's better to keep quiet and not say anything, as speaking may lead to trouble or problems.
Second turn: equivalent of the meaning in English: Silence is golden
Third turn: Recheck of the translation: Silence is golden


In [28]:

darija_idiom = "طلع تاكول الكرموس, نزل شكون قالها ليك"
print("Literal translation: Come upstairs and have some figs, get downstairs who told you so!.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: Come upstairs and have some figs, get downstairs who told you so!.

prompt's literal translation: Go up and eat the figs, come down who told you that.
prompt's explanation of original idiom in English: This idiom means "to take the blame for something you didn't do."

Simple equivalent translation: He went to eat the watermelon, and someone came and told you.
Purpose and target translation: He went out to eat cherries, and someone came down and told you.
Culture specefic translation: Went to eat some popcorn, came back, guess who said it to you

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is being told to do something, but then when they do it, they are criticized or blamed for it. It implies a sense of confusion or contradiction in the instructions given.
Second turn: equivalent of the meaning in English: Damned if you do, damned if you don't
Third turn: Recheck of the translation: Damned if you do,

In [29]:

darija_idiom = "ماديرش يدك في غيران باش مايعضوكش الحنوشة"
print("Literal translation: Don't stick your hand in holes in the ground so that you don't get bitten by snakes.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")



Literal translation: Don't stick your hand in holes in the ground so that you don't get bitten by snakes.

prompt's literal translation: Don't put your hand in the fire so it doesn't bite you.
prompt's explanation of original idiom in English: This idiom means "Don't meddle in other people's business so they don't bite you." It is a warning against interfering in matters that do not concern you to avoid potential negative consequences.

Simple equivalent translation: Don't put your hand in the beehive if you don't want to get stung.
Purpose and target translation: Don't put your hand in the beehive so you won't get stung.
Culture specefic translation: Don't put your hand in the cookie jar or you might get bitten.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means to avoid getting involved in a risky or dangerous situation to prevent negative consequences.
Second turn: equivalent of the meaning in English: An ounce of prevention is worth a 

In [30]:

darija_idiom = "ماديرش راسك في نخالة باش ماينقبوكش الدجاج"
print("Literal translation: Don't put your face in bran if you don't want to get pecked by the hens.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")

Literal translation: Don't put your face in bran if you don't want to get pecked by the hens.

prompt's literal translation: Don't put your head in the chaff so the chickens don't peck you.
prompt's explanation of original idiom in English: This idiom means "Don't involve yourself in matters that don't concern you to avoid getting into trouble."

Simple equivalent translation: Don't bury your head in the sand so the chickens don't peck you.
Purpose and target translation: Don't put your head in the chicken feed so the chickens won't peck at you.
Culture specefic translation: Don't bury your head in the sand so they don't peck you like chickens.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means to not involve yourself in matters that don't concern you, in order to avoid unnecessary trouble or conflict.
Second turn: equivalent of the meaning in English: Mind your own business
Third turn: Recheck of the translation: Mind your own business


In [31]:

darija_idiom = "دابزو يا الفيران على خميرة الجيران"
print("Literal translation: The mice are fighting over the neighbor's yeast.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")



Literal translation: The mice are fighting over the neighbor's yeast.

prompt's literal translation: The dogs are barking at the neighbors' dough.
prompt's explanation of original idiom in English: This idiom means "to act like a mouse in the neighbor's dough," which implies being nosy or meddling in other people's business.

Simple equivalent translation: The fox is guarding the neighbors' dough.
Purpose and target translation: He's like a fox guarding the neighbors' dough.
Culture specefic translation: Pizza or burgers at the neighbors' party

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that people are talking about things that are none of their business or that they are gossiping about others.
Second turn: equivalent of the meaning in English: To talk out of turn
Third turn: Recheck of the translation: To speak out of turn


In [32]:

darija_idiom = "اللي قال العصيدة باردة، ادير يدو فيها"
print("Literal translation: If one says the porridge is cold, put his hand in it.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")




Literal translation: If one says the porridge is cold, put his hand in it.

prompt's literal translation: Who said the porridge is cold, put his hand in it.
prompt's explanation of original idiom in English: This idiom means that the person who criticized something should try to improve it themselves.

Simple equivalent translation: If someone says the porridge is cold, put their hand in it.
Purpose and target translation: Whoever says the porridge is cold, should stir it themselves.
Culture specefic translation: Whoever said porridge is cold, should stir it themselves.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is confident in their ability to handle a situation or solve a problem.
Second turn: equivalent of the meaning in English: Don't knock it until you try it.
Third turn: Recheck of the translation: To have a trick up one's sleeve


In [33]:

darija_idiom = "شحال من الفيل قرصاتوا عقرب"
print("Literal translation: So many elephants have been stung by scorpions.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")






Literal translation: So many elephants have been stung by scorpions.

prompt's literal translation: How many bites can a scorpion give to an elephant?
prompt's explanation of original idiom in English: This idiom means that someone is facing a problem or challenge that is much smaller or less significant than what they are capable of handling.

Simple equivalent translation: How many times can a scorpion sting an elephant?
Purpose and target translation: How many times can a scorpion sting an elephant?
Culture specefic translation: How many times the elephant got stung by a scorpion

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is asking a question that has an obvious or impossible answer. It is used to point out when someone is asking a nonsensical or irrelevant question.
Second turn: equivalent of the meaning in English: Asking for the moon
Third turn: Recheck of the translation: Asking for the moon


In [34]:


darija_idiom = "الدنيا دراعة كل واحد كيلبسها ساعة"
print("Literal translation: The world is a sleeve, everyone wears it for a while.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")




Literal translation: The world is a sleeve, everyone wears it for a while.

prompt's literal translation: The world is a sleeve, everyone wears it for a while.
prompt's explanation of original idiom in English: This idiom means that everyone has their own way of dealing with life's challenges or difficulties.

Simple equivalent translation: The world is a sleeve, everyone wears it for an hour.
Purpose and target translation: The world is a slippery place, everyone wears it for an hour.
Culture specefic translation: The world is like a wristwatch, everyone wears it for a while.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that everyone has their time in the spotlight or their moment of fame, but eventually, it will pass and someone else will take their place.
Second turn: equivalent of the meaning in English: Different strokes for different folks
Third turn: Recheck of the translation: Different strokes for different folks.


In [35]:


darija_idiom = "الزين إلا زين الفعال"
print("Literal translation: The beauty is only in the actions.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")




Literal translation: The beauty is only in the actions.

prompt's literal translation: The good is only in doing good.
prompt's explanation of original idiom in English: This idiom means "the good is only in doing good deeds." It emphasizes the importance of performing good actions and deeds.

Simple equivalent translation: Actions speak louder than words.
Purpose and target translation: Beauty lies in good deeds.
Culture specefic translation: All that glitters is not gold

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that good deeds should be done without expecting anything in return.
Second turn: equivalent of the meaning in English: "Virtue is its own reward."
Third turn: Recheck of the translation: "Virtue is its own reward."


In [36]:

darija_idiom = "الحمار مخبي ووذنيه برّا"
print("Literal translation: The donkey is hiding but his ears are sticking out in plain sight.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")




Literal translation: The donkey is hiding but his ears are sticking out in plain sight.

prompt's literal translation: The donkey is stubborn and its ears are outside.
prompt's explanation of original idiom in English: This idiom means that someone is pretending to be innocent or unaware of something, while in reality they are fully aware and involved.

Simple equivalent translation: The donkey hides its ears outside.
Purpose and target translation: The donkey hides its ears outside.
Culture specefic translation: The donkey is stubborn and its ears are outside.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is stubborn and refuses to listen to reason or advice.
Second turn: equivalent of the meaning in English: The cat is out of the bag
Third turn: Recheck of the translation: To feign ignorance


In [37]:
darija_idiom = "الفقيه دحمان، قرا البرة سبع ايام، وقاليهم الخط عيان"
print("Literal translation: The jurist Dahman read the letter for seven days and declared the handwriting was poor.\n")

print(f"prompt's literal translation: {multiple_turns(darija_idiom)[0]}")
print(f"prompt's explanation of original idiom in English: {meaning(darija_idiom)}\n")


print(f"Simple equivalent translation: {ohne(darija_idiom)}")
print(f"Purpose and target translation: {culture(darija_idiom)}")
print(f"Culture specefic translation: {dynamic(darija_idiom)}\n")

print("multiple Turns translation:")
print(f"First turn: Literal translation's possible meaning: {multiple_turns(darija_idiom)[1]}")
print(f"Second turn: equivalent of the meaning in English: {multiple_turns(darija_idiom)[2]}")
print(f"Third turn: Recheck of the translation: {multiple_turns(darija_idiom)[3]}")


Literal translation: The jurist Dahman read the letter for seven days and declared the handwriting was poor.

prompt's literal translation: The wise man, Dhahman, read the letter for seven days, and told them the line is clear.
prompt's explanation of original idiom in English: This idiom means that someone has put in a lot of effort and time into something, only to realize that it was all in vain.

Simple equivalent translation: The scholar Dahman read the Quran for seven days, and then told them the line was crooked.
Purpose and target translation: The scholar Dahman read the Quran for seven days, and then told them the line was blurry.
Culture specefic translation: The scholar Dahman, read the camel hump for seven days, and told them the line is clear.

multiple Turns translation:
First turn: Literal translation's possible meaning: This idiom means that someone is taking a long time to understand or make a decision about something that should be obvious or straightforward.
Second tu